In [2]:
import pandas as pd
from pandas import DataFrame

In [6]:
df_tennis = pd.read_csv('C:/Users/HP/Documents/AIML LAB/Lab3.csv')

In [7]:
df_tennis

,Day,Outlook,Temperature,Humidity,Wind,Play Tennis
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Weak,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Strong,Yes


In [8]:
attribute_names = list(df_tennis.columns)

In [9]:
attribute_names

['Day', 'Outlook', 'Temperature', 'Humidity', 'Wind', 'Play Tennis']

In [10]:
attribute_names.remove('Day')
attribute_names.remove('Play Tennis')

In [11]:
attribute_names

['Outlook', 'Temperature', 'Humidity', 'Wind']

In [12]:
def entropy_of_list(lst):
    from collections import Counter
    count = Counter(x for x in lst)
    num_instances = len(lst)*1
    probs = [x/num_instances for x in count.values()]
    return entropy(probs)

In [13]:
def entropy(probs):
    import math
    return sum([-prob*math.log(prob,2)for prob in probs])

In [14]:
total_entropy = entropy_of_list(df_tennis['Play Tennis'])

In [15]:
def information_gain(df,split_attribute_name,target_attribute_name,trace = 0):
    df_split = df.groupby(split_attribute_name)
    nobs = len(df.index)*1
    df_agg_ent = df_split.agg({target_attribute_name:[entropy_of_list, lambda x: len(x)/nobs]})
    df_agg_ent.columns = ['entropy','propobservations']
    new_entropy = sum(df_agg_ent['entropy']*df_agg_ent['propobservations'])
    old_entropy = entropy_of_list(df[target_attribute_name])
    print(split_attribute_name,'IG:',old_entropy-new_entropy)
    return old_entropy - new_entropy

In [16]:
def ID3(df,target_attribute_name,attribute_names,default_classs=None):
    from collections import Counter
    count = Counter(x for x in df[target_attribute_name])
    if len(count) == 1:
        return next(iter(count))
    elif df.empty or (not attribute_names):
        return default_class
    else:
        default_class = max(count.keys())
        gain = [information_gain(df,attr,target_attribute_name)for attr in attribute_names]
        print()
        index_of_max = gain.index(max(gain))
        best_attr = attribute_names[index_of_max]
        tree = {best_attr:{}}
        remaining_attribute_names = [i for i in attribute_names if i!=best_attr]
        for attr_val,data_subset in df.groupby(best_attr):
            subtree = ID3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
            tree[best_attr][attr_val] = subtree
            
        return tree

In [17]:
from pprint import pprint
tree = ID3(df_tennis,'Play Tennis',attribute_names)
print("\n\n The resultant Decision tree is: \n")
pprint(tree)

Outlook IG: 0.2467498197744391
Temperature IG: 0.029222565658954647
Humidity IG: 0.15183550136234136
Wind IG: 0.04812703040826927

Temperature IG: 0.01997309402197489
Humidity IG: 0.01997309402197489
Wind IG: 0.4199730940219749

Temperature IG: 0.2516291673878229
Humidity IG: 0.2516291673878229

Humidity IG: 1.0

Temperature IG: 0.5709505944546686
Humidity IG: 0.9709505944546686
Wind IG: 0.01997309402197489



 The resultant Decision tree is: 

{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': {'Temperature': {'Cool': 'No',
                                                          'Mild': {'Humidity': {'High': 'No',
                                                                                'Normal': 'Yes'}}}},
                               'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
